<a href="https://colab.research.google.com/github/YasiruMM/Medicine-Prediction-Grp-22/blob/Diabetics_-Data_-Cleaning/Diabetics_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Imports**#

In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = 'Structured_Diabetics.xlsx'
df = pd.read_excel(file_path)

df.head()

,Disease Category,Drug Category,Drug Name,Dosage,Retail Price,Purchase Price,Sales,Date
0,Diabetics,ALPHA GLUCO,DIABOSE 50MG,50MG,2020603.2,1737412.6,3427.0,2.24
1,Diabetics,ALPHA GLUCO,GLUCOBAY 100MG,100MG,954.0,1514.0,1.0,2.24
2,Diabetics,ALPHA GLUCO,GLUCOBAY 100MG,100MG,421702.5,360710.5,753.0,2.24
3,Diabetics,ALPHA GLUCO,GLUCOBAY 50MG,50MG,187.6,327.6,0.0,2.24
4,Diabetics,ALPHA GLUCO,GLUCOBAY 50MG,50MG,13511.0,21434.4,52.0,2.24


#**Extract Unique pairs of drug names and categories**#

In [3]:
pairs = df.groupby('Drug Category')['Drug Name'].unique().to_dict()
print(pairs)

{'ALPHA GLUCO': array(['DIABOSE 50MG', 'GLUCOBAY 100MG', 'GLUCOBAY 50MG'], dtype=object), 'SULFON': array(['AMARYL 1MG', 'AMARYL 2MG', 'AMARYL 3MG', 'AZUKON MR 30MG',
       'CONSUCON 80MG', 'DAONIL 5MG', 'DIAMICRON 80MG',
       'DIAMICRON MR 30MG', 'DIAMICRON MR 60MG', 'DIANORM 80MG',
       'DIANORM OD 30', 'DIANORM OD 60', 'DIAPRIDE 1MG', 'DIAPRIDE 2MG',
       'DIATICA 80MG', 'DIAZIDE TAB 40MG', 'DIAZIDE TAB 80MG',
       'EUGLIM 2MG', 'EUGLIM 4MG', 'EUGLIZIP 5MG', 'GD-CARE 5MG',
       'GETRIL 1MG', 'GETRIL 2MG', 'GETRIL 3MG', 'GETRIL 4MG',
       'GLEMAZ 2MG', 'GLEMAZ 4MG', 'GLIBENCOMIDE 5MG (INGA)',
       'GLICAD 40MG', 'GLICLAZIDE 40MG', 'GLICLAZIDE 40MG (IPCA)',
       'GLICLAZIDE 80MG', 'GLICLAZIDE 80MG (SPMC)',
       'GLICLAZIDE 80MG SPMC', 'GLICLAZIDE MR 30MG',
       'GLICLAZIDE MR 60MG (IPCA)', 'GLIDABET 80MG', 'GLIMEPRIDE 2MG',
       'GLIMEPRIDE 2MG (USV)', 'GLIMIPRIDE 4MG', 'GLIPIZ  5MG',
       'GLIPIZIDE 5MG (USV)', 'GLITROL CR 30MG', 'GLITROL CR 60MG',
       'GL